# Replace partition

Caso de uso: Temos um dado onde é interessante substituir o conteúdo de um "pedaço" por vez, é necessario que o dataset seja particionado pelo campo sendo filtrado.
Se aplica bem em cargas contábeis normalmente mensais onde temos que carregar/recarregar um período contábil de uma vez. Nesse exemplo 


In [ ]:
# Exemplo simplista do uso de replacewhere

# https://dwgeek.com/replace-pyspark-dataframe-column-value-methods.html/
from pyspark.sql.functions import *


# Gerando dados ficticios
data = []
for ano in range(2020, 2023):
  for id in range (100):
    data.append ((ano, (ano - 2020) * 100 + id, str(id)))
df = spark.createDataFrame(data, ["year", "id", "d_id"])
df.show()


# Substituindo dados de 2020

df2020 = df.filter(col("year") == 2020)

df2020.write.format("delta") \
  .mode("overwrite") \
  .partitionBy("year") \
  .option("replaceWhere", "year = 2020") \
  .save("/tmp/teste2")


# Substituindo dados de 2021

df2021 = df.filter(col("year") == 2021)

df2021.write.format("delta") \
  .mode("overwrite") \
  .partitionBy("year") \
  .option("replaceWhere", "year = 2021") \
  .save("/tmp/teste2")

df2 = spark.read.format("delta").load("/tmp/teste2")
df2.show(1000)




In [ ]:
cleansed_dest_path = "my/dest/path"

In [ ]:
      # Esse código pega a ultima "versão" de um ano mes, ou seja, para cada ano mês 'olha" 
      # em todos os arquivos que foram selecionados e pega o ultimo. 
      # Para cada mês considera os dados apenas do  a ultima data de ingestão. 
      # Coloquei dentro do Loop para evitar que esse dataframe cresça absurdamente.
      #
      # Exemplo:
      #  Ingestão      AnoMes     Contagem
      #  28/02         202101     2200
      #  28/02         202102     3200
      #  08/03         202102     1000
      #  08/03         202103     100
      #  09/03         202102     1100
      #  09/03         202103     110
      #  Seleciona:
      #  28/02         202101     2200
      #  09/03         202102     1100
      #  09/03         202103     110

In [ ]:
last_exec_date = date.today() # Só um exemplo, Usar para isso um sistema de controle de mudança de data
folder = "path/to/data"
df = spark.read.json(folder).filter(col("DateIngest") >= last_exec_date)

In [ ]:
# ajusta os tipos dos dados
df = df.withColumn("vl_teste", col("vl_teste").cast(DoubleType()))\
       .withColumn("qt_teste", col("qt_teste").cast(DoubleType()))\
       .withColumn('ANO_MES', substring('Data', 1,6)) \


In [ ]:
# obtem uma lista de anos meses sendo substituidos
anos_meses = df2.groupby("ANO_MES", 'DateIngest').count().collect()

 
for ano_mes in anos_meses:
  print("Substituindo ano mes: {0} ".format(ano_mes.ANO_MES))
  df2.filter(col("ANO_MES") == ano_mes.ANO_MES) \
    .write.format("delta") \
    .mode("overwrite") \
    .partitionBy("ANO", "ANO_MES") \
    .option("mergeSchema", "true") \
    .option("replaceWhere", "ANO_MES = {0}".format(ano_mes.ANO_MES)) \
    .save(cleansed_dest_path)